<a href="https://colab.research.google.com/github/satishgc227/Derby-project/blob/main/Deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.metrics import Precision
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
import numpy as np
import keras
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
#Load the dataset
cdf=pd.read_csv("/content/edited complete.csv",encoding='latin',nrows=50000)

In [3]:
cdf.isnull().sum()

track_id               0
race_date              0
race_number            0
program_number         0
trakus_index           0
latitude               0
longitude              0
distance_id            0
course_type            0
track_condition        0
run_up_distance        0
race_type              0
purse                  0
post_time              0
weight_carried         0
jockey                 0
odds                   0
position_at_finish     0
dtype: int64

In [4]:
cdf=cdf.dropna()

In [5]:
cdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   track_id             50000 non-null  object 
 1   race_date            50000 non-null  object 
 2   race_number          50000 non-null  int64  
 3   program_number       50000 non-null  object 
 4   trakus_index         50000 non-null  int64  
 5   latitude             50000 non-null  float64
 6   longitude            50000 non-null  float64
 7   distance_id          50000 non-null  int64  
 8   course_type          50000 non-null  object 
 9   track_condition      50000 non-null  object 
 10  run_up_distance      50000 non-null  int64  
 11  race_type            50000 non-null  object 
 12  purse                50000 non-null  int64  
 13  post_time            50000 non-null  int64  
 14  weight_carried       50000 non-null  int64  
 15  jockey               50000 non-null 

In [6]:
# Encode ordinal columns ( categorical values with dtype-object)

le=LabelEncoder()
obj=cdf.select_dtypes(include='object').columns
for f in obj:
    cdf[f] = le.fit_transform(cdf[f].astype(str))
print(cdf.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   track_id             50000 non-null  int64  
 1   race_date            50000 non-null  int64  
 2   race_number          50000 non-null  int64  
 3   program_number       50000 non-null  int64  
 4   trakus_index         50000 non-null  int64  
 5   latitude             50000 non-null  float64
 6   longitude            50000 non-null  float64
 7   distance_id          50000 non-null  int64  
 8   course_type          50000 non-null  int64  
 9   track_condition      50000 non-null  int64  
 10  run_up_distance      50000 non-null  int64  
 11  race_type            50000 non-null  int64  
 12  purse                50000 non-null  int64  
 13  post_time            50000 non-null  int64  
 14  weight_carried       50000 non-null  int64  
 15  jockey               50000 non-null 

In [7]:
# Take subset of dataset and build a deep learning model
fdf = cdf.head(1000)
data = fdf.drop(['track_id','program_number\xa0','race_date','latitude\xa0','longitude\xa0','distance_id\xa0','position_at_finish\xa0'],axis=1)
target = fdf['position_at_finish\xa0']


In [13]:
# Scale the data (transforming  data so that it fits within a specific scale, like 0-100 or 0-1)

ss = StandardScaler()
data = pd.DataFrame(ss.fit_transform(data))
#print(data.shape, target.shape)

# Split the dataset

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size = 0.8, test_size = 0.2,
                                               random_state = 1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(800, 11) (200, 11) (800,) (200,)


In [14]:
model=Sequential()
model.add(Dense(11,activation='relu',input_dim=11)) #input layer
model.add(Dropout(0.1))
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='softmax'))
model.summary()
print(model.summary,'\n')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 11)                132       
                                                                 
 dropout_2 (Dropout)         (None, 11)                0         
                                                                 
 dense_4 (Dense)             (None, 10)                120       
                                                                 
 dropout_3 (Dropout)         (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 263
Trainable params: 263
Non-trainable params: 0
_________________________________________________________________
<bound method Model.summary of <keras.engine.sequential.

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
#Applying optimizers and callbacks
optimizer=keras.optimizers.Adam(learning_rate=0.0001)
callback=EarlyStopping(monitor='val_loss', min_delta=0.0001,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False)

#Compiling the model
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])


#Fitting the model
history=model.fit(X_train,y_train,batch_size=200, epochs=100,
                    validation_data = (X_test,y_test), verbose=1, callbacks=callback)



Epoch 1/100
4/4 [==============================] - 1s 60ms/step - loss: 1.4698 - accuracy: 0.0000e+00 - val_loss: 1.3221 - val_accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 13ms/step - loss: 1.5427 - accuracy: 0.0000e+00 - val_loss: 1.2859 - val_accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 1.5214 - accuracy: 0.0000e+00 - val_loss: 1.2499 - val_accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 1.4334 - accuracy: 0.0000e+00 - val_loss: 1.2138 - val_accuracy: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 10ms/step - loss: 1.4955 - accuracy: 0.0000e+00 - val_loss: 1.1780 - val_accuracy: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 1.3532 - accuracy: 0.0000e+00 - val_loss: 1.1424 - val_accuracy: 0.0000e+00
Epoch 7/100
4/4 [==============================] - 0s 10ms/step - loss: 1.3175 - accuracy: 0.0000e+00 - val_loss